<a href="https://colab.research.google.com/github/redwon007/JobShopPSO/blob/main/3x3_jobShop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
from copy import deepcopy
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
from datetime import datetime
import math
from random import randint, random

In [ ]:
nbMachines = input("nbMachines?")
nbJobs = input("nbJobs?")
nbDataSet = int(input("nbDataSet?"))

path = "tai" + nbMachines + "_" + nbJobs + ".txt"
fichier = open(path, "r")

for i in range(34*(nbDataSet-1) + 3 ):
    line = fichier.readline()
    
times = []
for i in range(15):
    line = [int(i) for i in fichier.readline().strip().split()]
    times.append(line)
    
line = fichier.readline()#read line "Machines\n"

machines = []
for i in range(15):
    line = [int(i) for i in fichier.readline().strip().split()]
    machines.append(line)
    
fichier.close()

In [ ]:
print(times)
print(machines)

[['69', '81', '81', '62', '80', '', '3', '38', '62', '54', '66', '88', '82', '', '3', '12', '88'], ['83', '51', '47', '15', '89', '76', '52', '18', '22', '85', '26', '30', '', '5', '89', '22'], ['62', '47', '93', '54', '38', '78', '71', '96', '19', '33', '44', '71', '90', '', '9', '21'], ['33', '82', '80', '30', '96', '31', '11', '26', '41', '55', '12', '10', '92', '', '3', '75'], ['36', '49', '10', '43', '69', '72', '19', '65', '37', '57', '32', '11', '73', '89', '12'], ['83', '32', '', '6', '13', '87', '94', '36', '76', '46', '30', '56', '62', '32', '52', '72'], ['29', '78', '21', '27', '17', '43', '14', '15', '16', '49', '72', '19', '99', '38', '64'], ['12', '74', '', '4', '', '3', '15', '62', '50', '38', '49', '25', '18', '55', '', '5', '71', '27'], ['69', '13', '33', '47', '86', '31', '97', '48', '25', '40', '94', '22', '61', '59', '16'], ['27', '', '4', '35', '80', '49', '46', '84', '46', '96', '72', '18', '23', '96', '74', '23'], ['36', '17', '81', '67', '47', '', '5', '51', '23

In [ ]:
machines = [ 
    [1,2,3],
    [2,1,3],
    [1,2,3]]
times = [
    [45,0,10],
    [10,20,34],
    [0,17,28]
]


In [ ]:
machines = [ 
    [1,2,3],
    [2,3,1],
    [3,1,2]]
times = [
    [3,9,2],
    [4,3,2],
    [9,3,4]
]


In [ ]:
def vectorSolToMatrixSol(vectorSol):
    matrixSol = [[] for i in range(len(machines))]
    operation = [1] * len(machines[0])
    for job in vectorSol:
        job = int(job)
        matrixSol[machines[job-1][operation[job-1]-1]-1].append((job,operation[job-1]))
        operation[job-1] += 1
    return(matrixSol)


In [ ]:
def createGantt(solution):
    gantt = deepcopy(solution)
    nbJobs = len(machines[0])
    toPlaceList = [1] * nbJobs
    endTimeList = [0] * nbJobs
    i,j = 0,0
    while(i<len(machines)):
    #for i in range(len(machines)):
        
        while(j<nbJobs):
        #for j in range(nbJobs):
            #print(i,j, " ", toPlaceList , " " , endTimeList)
            refresh = False
            for job, toPlace in enumerate(toPlaceList):
                
                if(solution[i][j][0] == job+1 and solution[i][j][1] == toPlace):
                    if(j==0 or gantt[i][j-1] != solution[i][j-1]):
                        #print(toPlaceList , " " , endTimeList)      
                        #print(i,j)
                        refresh = True
                        if(j == 0):
                            previous = 0
                        else:
                            previous = gantt[i][j-1][1]
                        startTime = max(previous, endTimeList[job])
                        endTime = startTime + times[job][solution[i][j][1]-1]
                        gantt[i][j] = (startTime, endTime)
                        toPlaceList[job] += 1
                        endTimeList[job] = endTime
                        #j = 0
            if(refresh == True):
                break
            j += 1
        i += 1
        j = 0    
        if(refresh == True):
            i=0
    return(gantt)
    

In [ ]:
def makespan(vectorSolution):
    gantt = createGantt(vectorSolToMatrixSol(vectorSolution))
    makespan = max(gantt[0][-1])
    for lastOp in gantt:     
      lastRelease = max(lastOp[-1])
      if(lastRelease > makespan):
        makespan = lastRelease
    return(makespan)

In [ ]:
def computeNextVelocity(velocity, pos, bPos, bbPos):
    nextVelocity = velocity + (bPos - pos) + (bbPos - pos)
    return nextVelocity

In [ ]:
def arrangeSol(sol, continuousSol):
    newSol = np.array([])
    while(np.size(sol) != 0):
        minIndex = np.where(continuousSol == np.amin(continuousSol))[0]
        
        for i in minIndex:
            newSol = np.append(newSol, sol[i])
        inds_to_delete = sorted(minIndex, reverse=True)
        for i in inds_to_delete:
            sol = np.delete(sol, i)
            continuousSol = np.delete(continuousSol, i)
    return(newSol)

In [ ]:
def firstNotSchedule(newPosition, position):
  scheduled = [0] * len(machines)
  for i, job in enumerate(position):
    if(np.count_nonzero(newPosition == job) == scheduled[int(job)-1]):
      return(job)
    else:
      scheduled[int(job)-1] +=1

In [ ]:
def newParticle(particle, pBestParticle, gBestParticle):
  newPosition = np.array([])
  #pIndex, pBIndex, gBIndex = 0, 0, 0
  for j in range(len(particle)):
    i = 0
    nbJobs = len(times)
    wc = randint(0, 10)
    wb = randint(0, 10-wc)
    #wg = 10-wc-wb
    #wc /= 10
    #wb/= 10
    #wg /= 10
    wc, wb, wg = 0.2, 0.3, 0.5
    u = random()
    if(u <= wc):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, particle))
    elif(u <= wb + wg):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, pBestParticle))
    elif(wc+wb < u):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, gBestParticle))

  return(newPosition)


In [ ]:
def createSol(nbJobs, nbMachines):
  sol = np.array([])
  for i in range(nbMachines):
    for j in range(1, nbJobs + 1):
      sol = np.append(sol, j)
  return(sol)

In [ ]:
def jobShopPSO():
    swarm = [{"position" : np.array([1,3,2,2,1,1,3,2,3]) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},

        {"position" : np.random.permutation(np.array([1,3,2,2,1,1,3,2,3])) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])}
        ]
    gBestParticle = swarm[0]["position"]
    gBestParticleCmax = makespan(gBestParticle)
    for i, particle in enumerate(swarm):
        particle["pBestParticle"] = particle["position"]
        particle["pBestParticleCmax"] = makespan(particle["pBestParticle"])
        particle["velocity"] = 0
        if(particle["pBestParticleCmax"] < gBestParticleCmax):
            gBestParticle = particle["position"]
            gBestParticleCmax = particle["pBestParticleCmax"]
        
    iteration = 0
    while(iteration < 20):
        iteration +=1
        print('\n\niteration', iteration)
        for i, particle in enumerate(swarm):
            particle["position"] = newParticle(particle["position"], particle["pBestParticle"], gBestParticle)
            pCmax = makespan(particle["position"])
            if(pCmax < particle["pBestParticleCmax"]):
                particle["pBestParticle"] = particle["position"]
                particle["pBestParticleCmax"] = pCmax
                if(pCmax < gBestParticleCmax):
                    gBestParticle = particle["position"]
                    gBestParticleCmax = pCmax
            print("particle",i," cmax = ", pCmax)
    return((gBestParticle, gBestParticleCmax))

In [ ]:
%%timeit
a = jobShopPSO()



iteration 1
particle 0  cmax =  127
particle 1  cmax =  134
particle 2  cmax =  134
particle 3  cmax =  99
particle 4  cmax =  134
particle 5  cmax =  124
particle 6  cmax =  137
particle 7  cmax =  113
particle 8  cmax =  137
particle 9  cmax =  137
particle 10  cmax =  147
particle 11  cmax =  99
particle 12  cmax =  137
particle 13  cmax =  154
particle 14  cmax =  137
particle 15  cmax =  137
particle 16  cmax =  137
particle 17  cmax =  134
particle 18  cmax =  137
particle 19  cmax =  127
particle 20  cmax =  134
particle 21  cmax =  137
particle 22  cmax =  120
particle 23  cmax =  137
particle 24  cmax =  137
particle 25  cmax =  109
particle 26  cmax =  124
particle 27  cmax =  137
particle 28  cmax =  99
particle 29  cmax =  127
particle 30  cmax =  134
particle 31  cmax =  134
particle 32  cmax =  102
particle 33  cmax =  99
particle 34  cmax =  137
particle 35  cmax =  134
particle 36  cmax =  124
particle 37  cmax =  124


iteration 2
particle 0  cmax =  134
particle 1  

In [ ]:
print(a)

(array([1, 3, 2, 2, 1, 1, 3, 2, 3]), 15)


In [ ]:
print(vectorSolToMatrixSol(a[0]))
print(a[1])
createGantt(vectorSolToMatrixSol(a[0]))

[[(1, 1), (3, 2), (2, 3)], [(2, 1), (1, 2), (3, 3)], [(3, 1), (2, 2), (1, 3)]]
15


[[(0, 3), (9, 12), (12, 14)],
 [(0, 4), (4, 13), (13, 17)],
 [(0, 9), (9, 12), (13, 15)]]

In [ ]:

df = pd.DataFrame([
    dict(Task="Job A", Start=convert_to_datetime(0), Finish=convert_to_datetime(3), Resource="Alex"),
    dict(Task="Job B", Start=convert_to_datetime(4), Finish=convert_to_datetime(5), Resource="Alex"),
    dict(Task="Job C", Start=convert_to_datetime(0), Finish=convert_to_datetime(2), Resource="Max")
])
#Maurice clerc PSO

NameError: name 'convert_to_datetime' is not defined

In [ ]:
def convert_to_datetime(x):
  return datetime.fromtimestamp(31536000+x*24*3600).strftime("%Y-%d-%m")

In [ ]:
num_tick_labels = np.linspace(start = 0, stop = 10, num = 11, dtype = int)
date_ticks = [convert_to_datetime(x) for x in num_tick_labels]

fig = ff.create_gantt(df)
fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
fig.write_html('first_figure.html', auto_open=True)
fig.show()